<a href="https://colab.research.google.com/github/snhuq/msci436.project/blob/main/Final436.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the Kaggle API client
!pip install kaggle

from google.colab import files

# Upload your kaggle.json file
files.upload()

# Create the .kaggle directory and move the kaggle.json file there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d alshival/nhtsa-complaints

# Unzip the downloaded dataset
!unzip -o nhtsa-complaints.zip

import pandas as pd

# Load datasets from the downloaded files
car_models = pd.read_csv('car_models.csv')
complaints = pd.read_csv('complaints.csv')
investigations = pd.read_csv('investigations.csv')
ratings = pd.read_csv('ratings.csv')
recalls = pd.read_csv('recalls.csv')

# Checking first few rows of each dataset to make sure they are loaded correctly
  print("Car Models Data")
  print(car_models.head())

  print("\nComplaints Data")
  print(complaints.head())

  print("\nInvestigations Data")
  print(investigations.head())

  print("\nRatings Data")
  print(ratings.head())

  print("\nRecalls Data")
  print(recalls.head())

# Inspect columns of each dataframe
  print("Car Models Columns:", car_models.columns)
  print("Complaints Columns:", complaints.columns)
  print("Investigations Columns:", investigations.columns)
  print("Ratings Columns:", ratings.columns)
  print("Recalls Columns:", recalls.columns)

# Rename columns to make it consistent
    complaints.rename(columns={'make': 'Make', 'model': 'Model', 'modelYear': 'ModelYear'}, inplace=True)
    recalls.rename(columns={'Make': 'Make', 'Model': 'Model', 'ModelYear': 'ModelYear'}, inplace=True)
    ratings.rename(columns={'Make': 'Make', 'Model': 'Model', 'ModelYear': 'ModelYear'}, inplace=True)
    investigations.rename(columns={'MAKE': 'Make', 'MODEL': 'Model', 'YEAR': 'ModelYear'}, inplace=True)

# Drop rows with missing values to maintain data quality
    car_models.dropna(inplace=True)
    complaints.dropna(inplace=True)
    investigations.dropna(inplace=True)
    ratings.dropna(inplace=True)
    recalls.dropna(inplace=True)

# Clean Investigations Data
# Replace '.' with NaN and then convert YEAR as integer
    investigations.replace('.', pd.NA, inplace=True)
    investigations['ModelYear'] = pd.to_numeric(investigations['ModelYear'], errors='coerce').fillna(0).astype(int)

# Fill or drop missing values based on context
  investigations.fillna({
    'CAMPNO': 'Unknown',
    'SUBJECT': 'Unknown',
    'SUMMARY': 'No summary available',
    'MFR_NAME': 'Unknown Manufacturer'
}, inplace=True)

# Verify that all columns have consistent data types
  print("\nInvestigations Data Types after Cleaning:")
  print(investigations.dtypes)

# Perform the merge operation using the common columns
  try:
      merged_data = pd.merge(complaints, recalls, on=['Make', 'Model', 'ModelYear'])
      merged_data = pd.merge(merged_data, ratings, on=['Make', 'Model', 'ModelYear'])
      merged_data = pd.merge(merged_data, investigations, on=['Make', 'Model', 'ModelYear'])
    print("\nMerged Data")
    print(merged_data.head())
  except KeyError as e:
    print(f"KeyError: {e}")
    print("Please verify the common column name and ensure it exists in all dataframes.")


In [ ]:
# Remove the duplicate entries
  merged_data.drop_duplicates(inplace=True)

# Handle missing values by filling with appropriate values or drop
  merged_data.fillna({
    'numberOfInjuries': 0,
    'numberOfDeaths': 0,
    'crash': False,
    'fire': False,
    'OverallRating': merged_data['OverallRating'].mode()[0] if 'OverallRating' in merged_data else 5,
    'SideCrashDriversideRating': merged_data['SideCrashDriversideRating'].mode()[0] if 'SideCrashDriversideRating' in merged_data else 'Not Rated'
}, inplace=True)

# Display the first few rows of the merged data after handling missing values
  print("\nMerged Data after Handling Missing Values")
  print(merged_data.head())

    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
    from imblearn.over_sampling import SMOTE
  import joblib
  import numpy as np

# Feature Engineering: Create additional features
    merged_data['incident_year'] = pd.to_datetime(merged_data['dateOfIncident']).dt.year
    merged_data['vehicle_age'] = merged_data['incident_year'] - merged_data['ModelYear']

# Select relevant features for the model
  features = merged_data[['Make', 'Model', 'ModelYear', 'numberOfInjuries', 'numberOfDeaths',
                        'crash', 'fire', 'OverallRating', 'SideCrashDriversideRating',
                        'vehicle_age']]

# One-Hot Encode categorical features
    categorical_features = ['Make', 'Model', 'OverallRating', 'SideCrashDriversideRating']
    encoder = OneHotEncoder(sparse_output=False, drop='first')  # Avoiding dummy variable trap
    encoded_categorical = encoder.fit_transform(features[categorical_features])
    encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features))

# Normalize numerical features
    numerical_features = ['ModelYear', 'numberOfInjuries', 'numberOfDeaths', 'vehicle_age']
    scaler = StandardScaler()
    scaled_numerical = scaler.fit_transform(features[numerical_features])
    scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_features)

# Combine the encoded categorical and scaled numerical features
  final_features = pd.concat([encoded_categorical_df, scaled_numerical_df], axis=1)

# Display the first few rows of the final features to ensure everything is correct
  print("\nFinal Features for Modeling")
  print(final_features.head())

# Assuming 'Recall' is the target variable in the merged_data
# Adjust the name of the target variable if necessary

# For demonstration, I'll create a dummy target variable
    merged_data['Recall'] = np.random.choice([0, 1], size=merged_data.shape[0])

# Split the data into training and test sets
    X = final_features
    y = merged_data['Recall']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Balancing dataset using SMOTE
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Hyperparameter tuning using GridSearchCV
  param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear']
}
  grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1')
  grid_search.fit(X_train_balanced, y_train_balanced)

# Best model from grid search
    best_model = grid_search.best_estimator_

# Cross-validation score
    cv_scores = cross_val_score(best_model, X_train_balanced, y_train_balanced, cv=5, scoring='f1')
  print(f"Cross-Validation F1 Score: {np.mean(cv_scores):.2f}")

# Train the best model
    best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
    y_pred = best_model.predict(X_test)

# Evaluate the model
  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

  print("\nAccuracy Score:")
  print(accuracy_score(y_test, y_pred))

# Model Performance Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

  print(f"Accuracy: {accuracy:.2f}")
  print(f"Precision: {precision:.2f}")
  print(f"Recall: {recall:.2f}")
  print(f"F1 Score: {f1:.2f}")

# Revised Cost Savings Estimation
# Assumptions:
# - Cost of a recall: $11000 per vehicle (Hyundai EV)
# - Cost of not recalling a faulty vehicle (e.g., lawsuits, fines): $3600000
# - Percentage of true positives that lead to actual recalls: 80%

    cost_of_recall = 11000
    cost_of_no_action = 3600000
    true_positives = np.sum((y_test == 1) & (y_pred == 1))
    false_negatives = np.sum((y_test == 1) & (y_pred == 0))

# Calculate savings only when recalls prevent lawsuits
  prevented_lawsuits = true_positives * cost_of_no_action * 0.8
  cost_of_recalls = true_positives * cost_of_recall
  cost_savings = prevented_lawsuits - cost_of_recalls
  print(f"Estimated Cost Savings: ${cost_savings:,.2f}")

# Time Efficiency remains the same
# Potential Safety Issues Identified remains the same
# Improvement in Customer Satisfaction remains the same

# Time Efficiency
# Assumptions:
# - Manual process cost: $520 per vehicle repair (average car repair cost)
# - Automated system cost: $470000 for the first year and $70000 per year after
# - Total predictions per month: 1000

    manual_process_cost_per_vehicle = 520
    automated_system_first_year_cost = 470000
    automated_system_annual_cost = 70000
    total_predictions_per_month = 1000

# Calculate the total cost for one year of manual vs automated
    manual_total_cost_first_year = total_predictions_per_month * 12 * manual_process_cost_per_vehicle
    automated_total_cost_first_year = automated_system_first_year_cost + automated_system_annual_cost

# Calculate cost savings after the first year
    manual_total_cost_second_year = total_predictions_per_month * 12 * manual_process_cost_per_vehicle
    automated_total_cost_second_year = automated_system_annual_cost

    first_year_savings = manual_total_cost_first_year - automated_total_cost_first_year
    second_year_savings = manual_total_cost_second_year - automated_total_cost_second_year

  print(f"First Year Cost Savings: ${first_year_savings:,.2f}")
  print(f"Second Year Cost Savings: ${second_year_savings:,.2f}")

# Potential Safety Issues Identified
    potential_safety_issues = np.sum(y_pred == 1)
  print(f"Potential Safety Issues Identified: {potential_safety_issues}")

# Improvement in Customer Satisfaction
# Assumptions:
# - Improvement in customer satisfaction per proactive recall: 5%
# - Number of proactive recalls per month: equal to the number of true positives

    improvement_per_recall = 0.05
    customer_satisfaction_improvement = true_positives * improvement_per_recall
  print(f"Estimated Improvement in Customer Satisfaction: {customer_satisfaction_improvement * 100:.2f}%")

# Save the best model and transformers for future use
  joblib.dump(best_model, 'recall_prediction_model_best.joblib')
  joblib.dump(encoder, 'encoder_best.joblib')
  joblib.dump(scaler, 'scaler_best.joblib')

  print("Best model, encoder, and scaler have been saved for future use.")

# Export Data for Power BI
# Model performance metrics
  model_performance = pd.DataFrame({
      'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
      'Score': [accuracy, precision, recall, f1]
  })

  # Cost analysis data
  cost_analysis = pd.DataFrame({
      'Year': ['First Year', 'Second Year'],
      'Manual Cost': [manual_total_cost_first_year, manual_total_cost_second_year],
      'Automated Cost': [automated_total_cost_first_year, automated_total_cost_second_year],
      'Cost Savings': [first_year_savings, second_year_savings]
  })

# Prediction metrics
  true_positives = np.sum((y_test == 1) & (y_pred == 1))
  false_positives = np.sum((y_test == 0) & (y_pred == 1))
  true_negatives = np.sum((y_test == 0) & (y_pred == 0))
  false_negatives = np.sum((y_test == 1) & (y_pred == 0))

  predictions = pd.DataFrame({
      'Metric': ['True Positives', 'False Positives', 'True Negatives', 'False Negatives'],
      'Count': [true_positives, false_positives, true_negatives, false_negatives]
  })

# Export Investigations Data for Power BI
    investigations.to_csv('investigations_cleaned.csv', index=False)

# Save dataframes to CSV
    model_performance.to_csv('model_performance.csv', index=False)
    cost_analysis.to_csv('cost_analysis.csv', index=False)
    predictions.to_csv('predictions.csv', index=False)

# Download the CSV files to your local machine
  from google.colab import files
    files.download('model_performance.csv')
    files.download('cost_analysis.csv')
    files.download('predictions.csv')
    files.download('investigations_cleaned.csv')



Merged Data after Handling Missing Values
   odiNumber           manufacturer  crash   fire  numberOfInjuries  \
0   11555100  Hyundai Motor America  False  False                 0   
1   11555100  Hyundai Motor America  False  False                 0   
2   11555100  Hyundai Motor America  False  False                 0   
3   11555100  Hyundai Motor America  False  False                 0   
4   11571460  Hyundai Motor America  False  False                 0   

   numberOfDeaths dateOfIncident dateComplaintFiled          vin  \
0               0     11/13/2023         11/14/2023  KM8KRDDF4RU   
1               0     11/13/2023         11/14/2023  KM8KRDDF4RU   
2               0     11/13/2023         11/14/2023  KM8KRDDF4RU   
3               0     11/13/2023         11/14/2023  KM8KRDDF4RU   
4               0     02/12/2024         02/12/2024  KM8KRDDF7RU   

                                          components  ... VehicleId  \
0                                  ELECTRICAL SYST

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>